# Introduction

This notebook seeks to create cross-validation of logistic regression with both built-in and custom metrics. 

This seemingly simple task turns out to be not that simple after all 🤪: For the custom f_beta scorer, the `cross_validate()` function automatically feeds binary predictions rather than predicted probabilities into the scorer, which makes the results inaccurate. 


There must be multiple solutions to the problem. Three of them were tested and compared in this notebook. 


1. A bit hacky and lazy: To create a subclass of LogisticRegression() and enforce the `predict_proba()` returns there.

2. Munual but ultimately customizable: Just create our own `cross_validate()` function. Then it can be customized to work with whatever self-defined estimators or processes we want to cross-validate. 

3. Define `needs_proba = True` in `make_scorer()` which is then fed into sklearn `cross_validate()`. Careful that even when we pass in `roc_auc_score()` to `make_scorer`, it won't automatically offer predicted probabilities if this is not specified (pls see details in the demo code and results below). 

In [1]:
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate, KFold, cross_val_predict, train_test_split
from sklearn.metrics import make_scorer, roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, fbeta_score
import numpy as np
import pandas as pd

### cross_validate()

Below are simple code doing cross-validation with LogisticRegression using two simple metrics, `roc_auc` and `gini`. 

In [19]:
# Load breast cancer dataset
data = load_breast_cancer()
X = data.data
y = data.target

# Create logistic regression model
model = LogisticRegression(max_iter = 5000)

# Set up k-fold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=123)

# Define scoring dictionary with multiple metrics
scoring = {
    'auc':  make_scorer(lambda y_true, y_pred_prob: 
                        roc_auc_score(y_true, y_pred_prob)),
    'gini': make_scorer(lambda y_true, y_pred_prob: 
                        2*roc_auc_score(y_true, y_pred_prob)-1),
}
# Perform cross-validation with multiple metrics
scores = cross_validate(model, X, y, cv=kfold, scoring=scoring)

predicts = cross_val_predict(model, X, y, cv=kfold, method = 'predict_proba')

# Print the scores for each metric
print("Gini scores:", scores['test_gini'])
print("AUC scores:", scores['test_auc'])
print("Mean Gini:       ", np.mean(scores['test_gini']))
print("Mean AUC scores: ", np.mean(scores['test_auc']))
print("Predicts:        ", predicts[:2])

Gini scores: [0.95121951 0.94887266 0.89294489 0.93452381 0.81503268]
AUC scores: [0.97560976 0.97443633 0.94647245 0.9672619  0.90751634]
Mean Gini:        0.9085187104778096
Mean AUC scores:  0.9542593552389048
Predicts:         [[1.00000000e+00 2.24267433e-13]
 [9.99997384e-01 2.61599435e-06]]


### Add a custom function
In many real-world business cases, we might want to utlize a customed metric to identify the optimal model for our own use case. It is very easy to implete with sklearn. 

Below is a (presumably) simple example where a function is created to calculate Fbeta score then feed into make_scorer() so it can be calculated in the cross-validation process. (for more on business cases, custom function and Fbeta, pls see ·metrics-fbeta.iynb` in the same notebook.)

In [20]:
def f_beta_scorer(y_true, y_prob, threshold, beta):
    """
    This function calculate Fbeta based on a threshold of 0.7, 
    which means cases with predicted probability higher than 0.7 
    will be judged as positive by the model. 
    It also allow user to define beta value for f_beta score.

    But the commented out code doesn't work, which means one-dimensional binary pred rather
    than predicted probabilities was fed into the scorer
    """
    #y_pred = (y_prob[:,1] >= threshold).astype(int)
    y_pred = (y_prob >= threshold).astype(int)
    return fbeta_score(y_true, y_pred, beta = beta)

# Update the scoring dictionary with the new custom metrics
scoring.update([ 
    ('f_beta', make_scorer(f_beta_scorer, threshold=0.7, beta=0.8))
])

# Perform cross-validation with multiple metrics
scores = cross_validate(model, X, y, cv=kfold, scoring=scoring)

# Print the scores for each metric
print("Gini scores:", np.round(scores['test_gini'], 2))
print("AUC scores:", np.round(scores['test_auc'], 2))
print("F_beta scores:", np.round(scores['test_f_beta'], 2))
print("Mean Gini:         {:.2f}".format(np.mean(scores['test_gini'])))
print("Mean AUC score:    {:.2f}".format(np.mean(scores['test_auc'])))
print("Mean F_beta score: {:.2f}".format(np.mean(scores['test_f_beta'])))

Gini scores: [0.95 0.95 0.89 0.93 0.82]
AUC scores: [0.98 0.97 0.95 0.97 0.91]
F_beta scores: [0.98 0.98 0.97 0.97 0.93]
Mean Gini:         0.91
Mean AUC score:    0.95
Mean F_beta score: 0.97


### Problem of the above calculations

The above code seem straightforward and reasonable but actually the result is not accurate. Because as you can see from the comment in the `f_beta_scorer` function, binary y_pred (from `predict`) rather than probability predictions (from `predict_proba`) were fed into the scorer function.  

### Solution 1: Subclass and the revised `predict()`
So now we have a problem to solve and there must be multiple ways to go around this. Below I will try two ways

First, a sort of cheaky solution which poped into my head first is to create a new class (maybe due to or the OOP studies I did a while a go, see `OOP and multiple models.ipynb` also in this repo)
* this new class is a subclass of LogisticRegression() and therefore inherit its abilities :P
* then enforce this new class's `predict()` method to return the predicted probilities (rather than binary predictions). 

Below pls see the code to impletement this simple idea.

In [14]:
class proba_logreg(LogisticRegression):
    def __init__(self):
        super().__init__(max_iter=5000)
    def predict(self, X):
        return LogisticRegression.predict_proba(self, X)
    
model_proba = proba_logreg()

def f_beta_scorer(y_true, y_prob, threshold, beta):
    """
    This function calculate Fbeta based on a threshold of 0.7, 
    which means cases with predicted probability higher than 0.7 
    will be judged as positive by the model.
    """
    y_pred = (y_prob >= threshold).astype(int)
    return fbeta_score(y_true, y_pred, beta = beta)

# Update the scoring dictionary with the new custom metrics
scoring = {
    'gini': make_scorer(lambda y_true, y_prob: 
                            2*roc_auc_score(y_true, y_prob[:,1])-1
                        ),
    'auc': make_scorer(lambda y_true, y_prob:
                            roc_auc_score(y_true, y_prob[:,1])
                        ),
    'f_beta': make_scorer(lambda y_true, y_prob:
                            f_beta_scorer(y_true, y_prob[:,1], threshold=0.7, beta=0.8)
                        )
}

scores_proba = cross_validate(model_proba, X, y, cv=kfold, scoring=scoring)

print("Gini scores:", np.round(scores_proba['test_gini'],2))
print("AUC scores:", np.round(scores_proba['test_auc'],2))
print("f-beta:", np.round(scores_proba['test_f_beta'],2))
print("Mean Gini (proba):           {:.2f}".format(np.mean(scores_proba['test_gini'])))
print("Mean f-beta (proba):         {:.2f}".format(np.mean(scores_proba['test_f_beta'])))
print("Mean AUC scores (proba):     {:.2f}".format(np.mean(scores_proba['test_auc'])))



Gini scores: [0.98 0.99 0.96 0.99 0.98]
AUC scores: [0.99 1.   0.98 0.99 0.99]
f-beta: [0.98 0.98 0.95 0.96 0.95]
Mean Gini (proba):           0.98
Mean f-beta (proba):         0.96
Mean AUC scores (proba):     0.99


### Solution 2: Do it manually

Of course, we could simply do the splits and calculate the relevant metrics manually like below. 

The manual process comes with the benefit of ultimate customizability:

1. it can be used to cross-validate any sklearn or customed algorithm class. (The first approach above is able to do this as well.) 
2. it can be easily applied from any stage of the modeling process. For example, if I have a customed class that does both feature engineering and model fit, using this approach I could either just test the fitted model acrossed kfold data or test the combined process by doing the kfold splits before feature engineering. 


In [10]:
# Set the number of folds
k = 5

# Split the data into k folds
kf = KFold(n_splits=k, shuffle=True)

# Initialize an array to store the results
results = {'gini': np.zeros(k), 'auc': np.zeros(k), 'f_beta': np.zeros(k)}

# Loop over the folds
for i, (train_index, test_index) in enumerate(kf.split(X)):
    
    # Get the training and testing data for this fold
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Fit the model on the training data
    model.fit(X_train, y_train)    
    
    # Evaluate the model on the testing data and store the results
    y_prob = model.predict_proba(X_test)[:, 1]
    results['gini'][i] = 2*roc_auc_score(y_test, y_prob) - 1
    results['auc'][i] = roc_auc_score(y_test, y_prob)
    results['f_beta'][i] = f_beta_scorer(y_test, y_prob, threshold=0.7, beta=0.8)

# Compute the mean and standard deviation of the results for each metric
mean_results = {metric: np.mean(results[metric]) for metric in results}
std_devs = {metric: np.std(results[metric]) for metric in results}

# Print the results
for metric in results:
    print("{}: Mean = {:.2f}, Std Dev = {:.2f}".format(metric.upper(), 
                                                        mean_results[metric], 
                                                        std_devs[metric]))


GINI: Mean = 0.99, Std Dev = 0.01
AUC: Mean = 0.99, Std Dev = 0.01
F_BETA: Mean = 0.96, Std Dev = 0.01


### Solution 3 and Discussions

As you probably noticed, the gini and auc scores from the two solutions seem much higher than what we received first. Therefore I suspect that when we use `make_scorer`, even when calculating auc score with the sklearn built in function `roc_auc_score`, `predict()` is the default input into our scorer. If that's the case, is there a way to adjust? Yes, all we need to do is to add `needs_proba = True` as an argument for the `make_scorer` function.

Upon studying the doc, I also found that for some built-in functions like `roc_auc_score`, we could pass its identifier into the scoring dictionary. For example, in the code below we could say `{'auto auc': 'roc_auc'}`, sk-learn recognizes `roc_auc` as an identifier for the roc_auc_score metric. It will automatically handle the selection of `predict_proba` to obtain the predicted probabilities for calculating the ROC AUC score. But this is not much use when we have more complex custom scorers. 

To summarize, if we only need to cross-validate an estimator's fit on certain metrics, either solution 1 or 3 will suffice. Solution 3 is the standard approach, while solution 1 might be useful for a slightly trickier task.

Last but not least, if we desire the utmost customization of the cross-validation process, solution 2 is likely the way to go.

In [18]:
# Load breast cancer dataset
data = load_breast_cancer()
X = data.data
y = data.target

# Create logistic regression model
model = LogisticRegression(max_iter = 5000)

# Set up k-fold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=123)

# Define scoring dictionary with multiple metrics
scoring = {
    'auc':  make_scorer(lambda y_true, y_pred_prob: 
                        roc_auc_score(y_true, y_pred_prob), needs_proba = True),
    'gini': make_scorer(lambda y_true, y_pred_prob: 
                        2*roc_auc_score(y_true, y_pred_prob)-1, needs_proba = True),
    'auc_auto': 'roc_auc'
}
# Perform cross-validation with multiple metrics
scores = cross_validate(model, X, y, cv=kfold, scoring=scoring)

#Print the scores for each metric
print("Gini scores:", scores['test_gini'])
print("AUC scores:", scores['test_auc'])
print("auto AUC scores:", scores['test_auc_auto'])
print("Mean Gini:       ", np.mean(scores['test_gini']))
print("Mean AUC scores: ", np.mean(scores['test_auc']))
print("Mean auto AUC scores: ", np.mean(scores['test_auc_auto']))



Gini scores: [0.98128968 0.99301366 0.96489996 0.98941799 0.97908497]
AUC scores: [0.99064484 0.99650683 0.98244998 0.99470899 0.98954248]
auto AUC scores: [0.99064484 0.99650683 0.98244998 0.99470899 0.98954248]
Mean Gini:        0.981541250535457
Mean AUC scores:  0.9907706252677286
Mean auto AUC scores:  0.9907706252677286
